In [ ]:
import cv2
from cv2 import imread
from matplotlib import pyplot as plt

img = imread("/dogimage.jpg")
imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

sift = cv2.SIFT_create()
keypoints, descriptors = sift.detectAndCompute(img, None)
sift_image = cv2.drawKeypoints(imgGray, keypoints, img)

plt.figure()
plt.imshow(sift_image, cmap="gray")
plt.savefig("output/sift_image")

In [ ]:
# https://www.codespeedy.com/find-similarity-between-two-images-in-python/
# https://stackoverflow.com/questions/50217364/sift-comparison-calculate-similarity-score-python
# https://stackoverflow.com/questions/43220408/measure-of-image-similarity-for-feature-matching